In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## gensim word2vec preparation

In [3]:
!pip install -U gensim
#we use this for the word2vec representations of each word
#remove if not using google colab. Instead, install from console

    100% |████████████████████████████████| 22.6MB 2.0MB/s 
Requirement not upgraded as not directly required: six>=1.5.0 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.11.0)
Requirement not upgraded as not directly required: scipy>=0.18.1 in /usr/local/lib/python3.6/dist-packages (from gensim) (0.19.1)
Requirement not upgraded as not directly required: numpy>=1.11.3 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.14.5)
    100% |████████████████████████████████| 1.4MB 16.1MB/s 
Requirement not upgraded as not directly required: requests in /usr/local/lib/python3.6/dist-packages (from smart-open>=1.2.1->gensim) (2.18.4)
    100% |████████████████████████████████| 133kB 21.4MB/s 
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /usr/local/lib/python3.6/dist-packages (from requests->smart-open>=1.2.1->gensim) (2018.4.16)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /usr/local/lib/python3.6/dist-packages (from r

use gensim to get the word2vec representations of each word  
reference here:  
https://www.quora.com/How-do-you-extract-vectors-from-word2vec-given-a-word

In [4]:
from gensim.models import KeyedVectors#, Word2Vec

#currently using gensim's default word2vec pretrained model, trained on googlenews data. 
#todo: train on wikiqa if the results are crap. 

w2vmodel = KeyedVectors.load_word2vec_format('drive/Colab Notebooks/datasets/word2vec/GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)

word2vec first testing

In [5]:

# print(w2vmodel.similarity('woman','man'))
wvdict = w2vmodel.wv.vocab
#this is the list of all the words in the vocabulary

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


## Preprocessing input strings

In [6]:
import numpy as np

`a.toks` contains the questions (repeat question lines correspond to the number of candidate answer sentences)  
`b.toks` contains the answers. Each line is the answer to the question on the same line of `a.toks`  
`id` contains the id of the question at that line in `a.toks`  
`sim` contains the labels: for each block of repeated-questions: there is one line containing a `1` corresponding to the line in `b.toks` containing the correct answer to the same line in `a.toks`

https://github.com/castorini/data/tree/master/WikiQA

In [7]:
def read_data(path, readtype):
  with open(path) as f:
    lines = f.readlines()
    
  if readtype == "questions" or readtype == "answers":
    return lines
  elif readtype == "labels":
    return [int(line) for line in lines]

Load the data

In [8]:
bigpath = "drive/Colab Notebooks/datasets/QuestionAnswering/TrecQA/"
questions = read_data(bigpath+"/train-all/a.toks", "questions")
answers = read_data(bigpath+"/train-all/b.toks", "answers")
labels = read_data(bigpath+"/train-all/sim.txt", "labels")
#question_ids = read_data(bigpath+"/train/id.txt", "labels")
#questions

In [9]:
original_labels = labels
from sklearn.model_selection import train_test_split

questions, questions_val, labels, labels_val = train_test_split(questions, original_labels, test_size = 0.2, random_state = 12)
answers, answers_val, _, _ = train_test_split(answers, original_labels, test_size = 0.2, random_state = 12)

test to see if the splits worked

In [10]:
p = 66088
print(len(questions))
print(questions[p])
print(answers[p])
print(labels[p])

42733


IndexError: ignored

In [10]:
#resolve imbalanced data by oversampling positive results
n_positiveduplicates = 10

oversamp_questions = []
oversamp_answers = []
oversamp_labels = []
for i in range(len(questions)):
  if labels[i]==1:
    for n in range(n_positiveduplicates):
      oversamp_questions.append(questions[i])
      oversamp_answers.append(answers[i])
      oversamp_labels.append(labels[i])
      
      
questions += oversamp_questions
answers += oversamp_answers
labels += oversamp_labels

In [11]:
from sklearn.utils import shuffle
questions, answers, labels = shuffle(questions, answers, labels)

#### creating one-hot labels

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
labels_np = np.array(labels)
labels_np = np.expand_dims(labels_np, axis=1)

onehot = OneHotEncoder(n_values = 2)
onehot.fit(labels_np)
labels_np = onehot.transform(labels_np)

In [14]:
val_labels_np = np.array(labels_val)
val_labels_np = np.expand_dims(val_labels_np, axis=1)

onehot.fit(val_labels_np)
val_labels_np = onehot.transform(val_labels_np)

#### preparing word-level sequence embedding

This is to prepare for the Embedding layer

1. concatenate questions and answers into a list of `[question | answer]` pairs. This is a list of sequences.  
2. find max sequence length from this list  
3. the vocab size of the google news word2vec is 3 million words, the number of features is 300

The embedding_matrix:
1. let each word in `w2vmodel.wv.vocab.keys()` have an index
2. concatenate axis=1 all the word vectors in the vocabulary into embedding_matrix
3. then turn each string sequence into an array of numbers (each number represents the index of that word in the vocab)
4. pass the string sequences into the .fit as training input. The shape of this input should be (n_seqs, max_seq_length, n_features)
5. the embedding matrix will be the weights in the Embedding layer

In [15]:
vocab_size = 299567+1 #this is the SLIMmed googlenews w2v model's vocab count. #the last item is the 'all-zeroes' word vector, used for words that aren't in the w2vmodel

In [16]:
#make embedding matrix
embedding_matrix = np.zeros((vocab_size, 300))
for i in range(len(wvdict)):
    embedding_vector = w2vmodel.wv[w2vmodel.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


#### define some functions to preprocess data (both train and test)

In [17]:
def SEQ_max_length(sequences):
  return max( [len(qa.split()) for qa in sequences] )

In [18]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [19]:
def SEQ_token_and_pad(sequences, max_seq_length):
  encoded_seqs = []
  for qa in sequences:
    encoded_qa = []
    tokenized_qa = qa.split()
    for word in tokenized_qa:
      index_to_append = vocab_size - 1
      
      if word in wvdict.keys():
        index_to_append = w2vmodel.vocab[word].index
      encoded_qa.append(index_to_append)
    
    encoded_seqs.append(encoded_qa)
    
  
  padded_seqs = pad_sequences(encoded_seqs, maxlen = max_seq_length, padding="post", value = vocab_size - 1)
  return padded_seqs

Performing the necessary preprocessing on our data using our functions

In [20]:

max_question_length = SEQ_max_length(questions)
max_answer_length = SEQ_max_length(answers)
val_max_question_length = SEQ_max_length(questions_val)
val_max_answer_length = SEQ_max_length(answers_val)
max_hidden_length = max(max_question_length, max_answer_length, val_max_question_length, val_max_answer_length)

#version 1: getting separate questions and answers vectors (but still encoded in index form, not in pure word vector form)
padded_questions = SEQ_token_and_pad(questions, max_hidden_length)
padded_answers = SEQ_token_and_pad(answers, max_hidden_length)


do the same preprocessing on our validation set

In [21]:
val_padded_questions = SEQ_token_and_pad(questions_val, max_hidden_length)
val_padded_answers = SEQ_token_and_pad(answers_val, max_hidden_length)

## Model building and training

### Guesswork and experimentation



---


*If i'm doing the implementation in the 1506.06490 paper*  
**Todo**: 

- find length of the longest question and longest answer
- use these maxes for padding to shorter sentences
- in one big model, build 2 separate convnet submodels, one for questions, one for answers
- concatenate the outputs for the questions and answers convnets, and we obtain the **joint representation** `p` of each question-answer pair.

Each of these learned joint pairs is a single **timestep** in the LSTM part. We learn the sequence and semantic relationships between each answer-candidate sentence in the context paragraph, not between the words themselves. (the word-level patterns are learned in the Conv stage already)

for the LSTM stage, padding instead involves adding dummy qa pairs for each timestep without an answer. The max number of answer candidates in the dataset (and possible qa pairs, per question) is 30, so any Qs with fewer than 30 answer sentences will be padded with dummy pairs.


---



*my own simple-ass idea*  
**Todo:**

version 0a: `shit lstm model copped from some blog`  
https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

version 0b: `with learning rate, and fixed word tokenizations`

version 1a: `separate LSTM for question and answer, then Concatenate into a hidden layer which then is fed into classifier/Dense/whatever`

version 1b: `no LSTM for question and answer. Using Functional API. WORKS but OVERFITS.`

### Giang's idea

Giang's idea:

`doc question  
input_1 = Input(...)  
...  
output_1 = ...


doc answer  
input_2 = Input(...)  
...  
output_2 = ...

input_2 = Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_answer_length, trainable=False)  
  
output_2 = LSTM(100)(input_reshape_2)

hidden = Concatenate()([output_1, output_2])
classifier = Dense(1, activation="sigmoid")(hidden)
`

In [22]:
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, LSTM, Dense, Conv1D, MaxPooling1D, Dropout
from keras.optimizers import Adam

#using keras functional api

qumodel_input = Input(shape = (max_hidden_length,), name="questionsMainInput")
qumodel_embed = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)(qumodel_input)
qumodel_parse = Conv1D(32, 2)(qumodel_embed)
qumodel_parse = Dropout(0.29)(qumodel_parse)
qumodel_parse = MaxPooling1D(pool_size = 2)(qumodel_parse)

ansmodel_input = Input(shape = (max_hidden_length,), name="answersMainInput")
ansmodel_embed = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)(ansmodel_input)
ansmodel_parse = Conv1D(32,2)(ansmodel_embed)
ansmodel_parse = Dropout(0.29)(ansmodel_parse)
ansmodel_parse = MaxPooling1D(pool_size = 2)(ansmodel_parse)

combinedmodel = Concatenate()([qumodel_parse, ansmodel_parse])
print(combinedmodel.shape)

lstmcombinedmodel = LSTM(100)(combinedmodel)
classifier = Dense(32, activation='relu')(lstmcombinedmodel)
classifier = Dense(16, activation='relu')(classifier)
classifier = Dense(2, activation='softmax')(classifier)

final_model = Model(inputs = [qumodel_input, ansmodel_input], outputs = classifier)

final_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr = 0.003), metrics=['accuracy'])
print(final_model.summary())

(?, 29, 64)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
questionsMainInput (InputLayer) (None, 60)           0                                            
__________________________________________________________________________________________________
answersMainInput (InputLayer)   (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 300)      89870400    questionsMainInput[0][0]         
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 60, 300)      89870400    answersMainInput[0][0]           
_________________________________________________________________________________________________

## Fit the chosen model

Severely punish "no" answers because that's just lazy. The network should at least try to guess "yes" or "1" on at least some questions, and not settle for a 88% accuracy by answering no to everything.

add class_weights and try:

In [58]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', np.unique(labels), labels)
class_weights
#class_weights = np.array([0.5, 24.])

array([0.56809674, 4.17124785])

In [28]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="drive/Colab Notebooks/checkpts/weights.{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}.hdf5", monitor = "val_acc", save_best_only = True, mode="max")
callback_list = [checkpointer]

In [0]:
#final_model.fit([padded_questions, padded_answers], labels, epochs=5, batch_size=128, verbose=1, class_weight = class_weights, validation_split=0.2)

#model.fit([padded_questions, padded_answers], labels, batch_size=32,
 #         epochs=20, validation_split=0.1, class_weight = class_weights)
  
final_model.fit([padded_questions, padded_answers], labels_np, 
                epochs=30, 
                batch_size=128, 
                verbose=1, 
                validation_data = ([val_padded_questions, val_padded_answers], val_labels_np),
                callbacks = callback_list
               )

Train on 94283 samples, validate on 10684 samples
Epoch 1/30
88064/94283 [===========================>..] - ETA: 4s - loss: 0.4941 - acc: 0.7686

94283/94283 [==============================] - 71s 755us/step - loss: 0.4907 - acc: 0.7706 - val_loss: 0.4937 - val_acc: 0.7005
Epoch 2/30
54784/94283 [================>.............] - ETA: 32s - loss: 0.4063 - acc: 0.8190

94283/94283 [==============================] - 77s 812us/step - loss: 0.3936 - acc: 0.8267 - val_loss: 0.4253 - val_acc: 0.7736
Epoch 3/30
41600/94283 [============>.................] - ETA: 45s - loss: 0.3491 - acc: 0.8500

94283/94283 [==============================] - 76s 804us/step - loss: 0.3393 - acc: 0.8557 - val_loss: 0.4306 - val_acc: 0.7999
Epoch 4/30
37120/94283 [==========>...................] - ETA: 50s - loss: 0.3019 - acc: 0.8777

94283/94283 [==============================] - 76s 809us/step - loss: 0.2959 - acc: 0.8792 - val_loss: 0.3842 - val_acc: 0.8225
Epoch 5/30
35584/94283 [==========>...................] - ETA: 52s - loss: 0.2713 - acc: 0.8906

94283/94283 [==============================] - 77s 812us/step - loss: 0.2605 - acc: 0.8961 - val_loss: 0.3767 - val_acc: 0.8326
Epoch 6/30
 9856/94283 [==>...........................] - ETA: 1:24 - loss: 0.2472 - acc: 0.9018

In [0]:
final_model.save("cnnlstm-rollercoaster.h5")

## ...or load a checkpoint hdf5

(must compile model first!, then load in the weights hdf5 and start predicting/continue fitting)

In [23]:
checkptname = "weights.24-0.35-0.90.hdf5"
final_model.load_weights("drive/Colab Notebooks/checkpts/"+checkptname)

## Testing the trained model

#### Preparing the test data

In [25]:
# test_questions = ["Where was Tuan born","Where was Tuan born","Where was Tuan born","Where was Tuan born"]
# test_answers = ["Pigs dogs cats","Boo","Candy balls cake", "Tuan was born in Vietnam"]
# test_labels = [0, 0, 0, 1]



test_questions = read_data(bigpath+"/raw-test/a.toks", "questions")
test_answers = read_data(bigpath+"/raw-test/b.toks", "answers")
test_labels = read_data(bigpath+"/raw-test/sim.txt", "labels")
# question_ids = read_data(bigpath+"/dev/id.txt", "labels")

In [26]:
#test_seqs, max_test_seq_length = SEQ_make_qa_sequences_list(test_questions, test_answers)
padded_test_questions = SEQ_token_and_pad(test_questions, max_hidden_length)
padded_test_answers = SEQ_token_and_pad(test_answers, max_hidden_length)

#### Predicting some q, a pairs

In [27]:
test_preds = final_model.predict_on_batch([padded_test_questions, padded_test_answers])

In [0]:
print(test_preds)
np.argmax(test_preds*1000)

[[9.9997854e-01 2.1408780e-05]
 [9.9997807e-01 2.1898602e-05]
 [9.9996340e-01 3.6599067e-05]
 [9.9997723e-01 2.2740214e-05]]


0

In [38]:
i=77 #test example to check

# print(padded_test_seqs[1])
# print(padded_test_seqs[16])

print(test_preds[i])
print(test_questions[i])
print(test_answers[i])
print("correct label: "+str(test_labels[i]))

[9.9959809e-01 4.0195804e-04]
who is the president or chief executive of amtrak ?

amtrak is offering vouchers , instead of refunds on the current trip , to give itself a second chance to woo the customer .

correct label: 0
